This notebook contains data preparation pipeline for the [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) that was originaly collected in paper [Improving Recommendation Lists Through
Topic Diversification](http://www2.informatik.uni-freiburg.de/~cziegler/BX/WWW-2005-Preprint.pdf). Authors crawled [BookCrossing](https://www.bookcrossing.com/) website and collected data on 278 858 members and 1 157 112 ratings, both implicit and explicit, referring to 271 379 distinct ISBNs. Invalid ISBNs
were excluded from the outset.

# Setup



## Packages

Installing and importing packages. We will work with this nice python package called [isbnlib](https://github.com/xlcnd/isbnlib) that can be used to validate, clean, transform, hyphenate and get metadata for ISBN strings.

In [1]:
!pip install isbnlib

     |████████████████████████████████| 65 kB 1.5 MB/s 


In [2]:
import pandas as pd
import isbnlib
from tqdm import tqdm
import matplotlib.pyplot as plt

## Data

We download Book-Crossing dataset in CSV format, unzip it and load it into pandas DataFrame. During data loading, we need to set `encoding` parameter, because data are in `ISO-8859-1` encoding, not default `UTF-8`. Also, one book title contains quote, so we need to escape it, because quote is also used to encapsulate fileds in CSV file. Troubling title looks like this:

> `Peterman Rides Again: Adventures Continue with the Real \"J. Peterman\" Through Life &amp; the Catalog Business`

We need to explicitly set backslash (`\`) as escape chareacter.

In [3]:
!wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip

--2021-11-17 17:56:30--  http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
Resolving www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)... 132.230.105.133
Connecting to www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)|132.230.105.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘BX-CSV-Dump.zip’

BX-CSV-Dump.zip     100%[===================>]  24.88M  16.4MB/s    in 1.5s    

2021-11-17 17:56:33 (16.4 MB/s) - ‘BX-CSV-Dump.zip’ saved [26085508/26085508]



In [4]:
!unzip BX-CSV-Dump.zip

Archive:  BX-CSV-Dump.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:
users = pd.read_csv('BX-Users.csv', sep=';', encoding = "ISO-8859-1")
books = pd.read_csv('BX-Books.csv', sep=';', encoding = "ISO-8859-1", escapechar = "\\")
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding = "ISO-8859-1")

# Data cleaning



## Useless columns

Let's look at columns in `books` table if there are some useful information.

In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


We might use image in final application to show a preview of sugested books to a user. Or we could use it in multi-modal model as one of input for computing similarity between books. But for now, we will not need it, since we will do just simple proof-of-concept recommendation system.

In [7]:
books = books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


## Renaming columns

What we could do to make our life easier is to rename columns.

In [9]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication',
       'Publisher'],
      dtype='object')

In [10]:
books.columns = ['ISBN', 'Title', 'Author', 'Year', 'Publisher']

In [11]:
books.columns

Index(['ISBN', 'Title', 'Author', 'Year', 'Publisher'], dtype='object')

In [12]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [13]:
ratings.columns = ['User-ID', 'ISBN', 'Rating']

In [15]:
ratings.columns

Index(['User-ID', 'ISBN', 'Rating'], dtype='object')

In [16]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

Columns in `users` table are okey, we will keep them.